In [1]:
import pandas as pd
import numpy as np

employees = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'first_name': ['Ana', 'Bob', 'Cara', 'Dave', 'Eve', 'Frank', 'Grace', 'Hank', 'Iris', 'Jack'],
    'last_name': ['Smith', 'Jones', 'Brown', 'Wilson', 'Davis', 'Miller', 'Moore', 'Taylor', 'Anderson', 'Thomas'],
    'dept': ['Tech', 'Sales', 'Tech', 'Marketing', 'Tech', 'Sales', 'Marketing', 'Tech', 'Sales', None],
    'salary': [95000, 80000, 92000, 75000, 98000, 82000, 77000, 91000, 79000, None],
    'manager_id': [None, 1, 1, 2, 1, 2, 4, 3, 6, 7],
    'hire_date': ['2018-01-15', '2019-03-22', '2018-06-30', '2020-01-10', '2017-09-01', '2019-11-15', '2020-08-20', '2018-04-12', '2021-02-28', '2022-05-16']
})

departments = pd.DataFrame({
    'dname': ['Tech', 'Sales', 'Marketing', 'HR'],
    'budget': [500000, 300000, 200000, 150000]
})

In [2]:
-- Find departments where:
-- 1. Average salary > 85000 OR total salary > 250000
-- 2. Show: dept name, avg salary, total salary, employee count, salary range (max-min)
-- 3. Sort by total salary descending

SyntaxError: invalid syntax (2524046309.py, line 1)

In [53]:
emp = employees[['dept', 'salary', 'first_name']]
emp = emp[(emp['salary'].notna()) & (emp['dept'].notna())]
empie = emp.groupby('dept').agg(
    count = ('first_name' , 'count'),
    avg = ('salary', 'mean'),
    sumi = ('salary','sum'),
    maxi = ('salary','max'),
    mini = ('salary', 'min')
    )
empie = empie.reset_index()
empie = empie[(empie['avg'] > 85000) & (empie['sumi'] > 250000)]
empie['range'] = empie['maxi'] - empie['mini']
empie.sort_values('sumi')
empie

,dept,count,avg,sumi,maxi,mini,range
2,Tech,4,94000.0,376000.0,98000.0,91000.0,7000.0


-- Create a 'performance_tier' column based on:
-- Tier A: Salary > dept average AND hired before 2020
-- Tier B: Salary > dept average OR hired before 2020 (but not both)
-- Tier C: Everyone else
-- Then count employees in each tier by department

In [102]:
# group by dept 
emp = employees
boob = emp.groupby('dept').agg(
    average = ('salary' , 'mean')
    )
emp = emp.merge(boob, on='dept', how='inner')
emp['performance_tier'] = np.where((emp['salary'] > emp['average']) & (pd.to_datetime(emp['hire_date']).dt.year < 2020), 'Tier A', 
                          np.where(((emp['salary'] > emp['average']) & (pd.to_datetime(emp['hire_date']).dt.year >= 2020)) | 
                                   ((emp['salary'] < emp['average']) & (pd.to_datetime(emp['hire_date']).dt.year < 2020)), 
                                    'Tier B', 'Tier C'))
emp

,id,first_name,last_name,dept,salary,manager_id,hire_date,average,performance_tier
0,1,Ana,Smith,Tech,95000.0,NaN,2018-01-15,94000.000000,Tier A
1,2,Bob,Jones,Sales,80000.0,1.0,2019-03-22,80333.333333,Tier B
2,3,Cara,Brown,Tech,92000.0,1.0,2018-06-30,94000.000000,Tier B
3,4,Dave,Wilson,Marketing,75000.0,2.0,2020-01-10,76000.000000,Tier C
4,5,Eve,Davis,Tech,98000.0,1.0,2017-09-01,94000.000000,Tier A
5,6,Frank,Miller,Sales,82000.0,2.0,2019-11-15,80333.333333,Tier A
6,7,Grace,Moore,Marketing,77000.0,4.0,2020-08-20,76000.000000,Tier B
7,8,Hank,Taylor,Tech,91000.0,3.0,2018-04-12,94000.000000,Tier B
8,9,Iris,Anderson,Sales,79000.0,6.0,2021-02-28,80333.333333,Tier C


In [ ]:
-- For each department:
-- 1. Rank employees by salary (1 = highest)
-- 2. Calculate the salary difference between each employee and the next lower rank
-- 3. Show only employees ranked 1-3 with their salary gaps

In [141]:
emp = employees.copy()
emp= emp[emp['salary'].notna()]
emp['ranking']= emp.groupby('dept')['salary'].rank(ascending=False)
emp = emp.sort_values(['dept', 'ranking'])
emp['salary_shifted'] = emp['salary'].shift(-1)
emp['diff'] = emp['salary'] - emp['salary_shifted']
emp

,id,first_name,last_name,dept,salary,manager_id,hire_date,ranking,salary_shifted,diff
6,7,Grace,Moore,Marketing,77000.0,4.0,2020-08-20,1.0,75000.0,2000.0
3,4,Dave,Wilson,Marketing,75000.0,2.0,2020-01-10,2.0,NaN,NaN
5,6,Frank,Miller,Sales,82000.0,2.0,2019-11-15,1.0,80000.0,2000.0
1,2,Bob,Jones,Sales,80000.0,1.0,2019-03-22,2.0,79000.0,1000.0
8,9,Iris,Anderson,Sales,79000.0,6.0,2021-02-28,3.0,NaN,NaN
4,5,Eve,Davis,Tech,98000.0,1.0,2017-09-01,1.0,95000.0,3000.0
0,1,Ana,Smith,Tech,95000.0,NaN,2018-01-15,2.0,92000.0,3000.0
2,3,Cara,Brown,Tech,92000.0,1.0,2018-06-30,3.0,91000.0,1000.0
7,8,Hank,Taylor,Tech,91000.0,3.0,2018-04-12,4.0,NaN,NaN


-- Calculate for each department:
-- 1. Average tenure (years employed) as of 2025-07-18
-- 2. Turnover rate (% of employees hired after 2020)
-- 3. Average salary by hiring year
-- Present as a multi-column summary

Department | Avg Tenure | Turnover % | Avg Salary (Recent) | Avg Salary (Veteran)
Tech       | 5.2 years  | 25%        | $89,000            | $96,000
Sales      | 4.1 years  | 33%        | $78,000            | $82,000

In [351]:
# Copy original & Get hire year & Get today's date -> Calculate tenure years
emp = employees[employees['dept'].notna()].copy()
emp['hire_year'] = pd.to_datetime(emp['hire_date']).dt.year
today = pd.to_datetime('2025-07-18')
emp['tenure_years'] = (today - pd.to_datetime(emp['hire_date'])).dt.days / 365.25
grouped_tenure = emp.groupby('dept').agg({'tenure_years' : 'mean'})
# Get employees after 2020, count groups by dept & count all emps grouped by dept
recent_hires = emp[emp['hire_year'] > 2020]
filterfirst = recent_hires.groupby('dept').size()
total_per_dept = emp.groupby('dept').size()
# Get percentage by comparing small groups to big groups
turnover_pct = (filterfirst / total_per_dept * 100).fillna(0)
# Create copy
merged = grouped_tenure.copy()
# Assign series to column
merged['Turnover%'] = turnover_pct
# Since dept is the index and not a column, we assign the "index" instead of the values to all_depts
all_depts = merged.index
# Group by dept, get salary averages, reindex by the dept names we got earlier and fill NA values with 0
recent_salaries = recent_hires.groupby('dept')['salary'].mean().reindex(all_depts, fill_value=0)
merged['Avg Salary (Recent)'] = recent_salaries
# Same approach but for old salaries
old_salaries = (emp[emp['hire_year'] <= 2020]).groupby('dept')['salary'].mean().reindex(all_depts, fill_value=0)
merged['Avg Salary (Veteran)'] = old_salaries
merged

AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [335]:
-- Merge employees with departments, then:
-- 1. Calculate each employee's salary as % of department budget
-- 2. Flag employees whose salary is > 20% of dept budget
-- 3. Show department utilization (total salaries / budget)

SyntaxError: unterminated string literal (detected at line 2) (1409288967.py, line 2)

In [381]:
merged = employees.merge(departments, left_on=('dept'), right_on=('dname'))
merged['Salary%ofBudget'] = (merged['salary'] / merged['budget']) * 100
merged['FlagMoreThan20%'] = np.where(merged['Salary%ofBudget'] > 20, 'Yes', 'No')
deptUtil = (merged.groupby('dept')['salary'].sum() / merged.groupby('dept')['budget'].mean()) * 100
# 2 SOLUTIONS BELOW!
#EITHER this
merged['Department Utilization'] = merged['dept'].map(deptUtil)
#OR this with merge
#deptUtil_df = deptUtil.to_frame('Department Utilization')
#merged = merged.merge(deptUtil_df, on='dept')

# Round for readability
merged['Salary%ofBudget'] = merged['Salary%ofBudget'].round(1)
merged['Department Utilization'] = merged['Department Utilization'].round(1)

merged

,id,first_name,last_name,dept,salary,manager_id,hire_date,dname,budget,Salary%ofBudget,FlagMoreThan20%,Department Utilization
0,1,Ana,Smith,Tech,95000.0,NaN,2018-01-15,Tech,500000,19.0,No,75.2
1,2,Bob,Jones,Sales,80000.0,1.0,2019-03-22,Sales,300000,26.7,Yes,80.3
2,3,Cara,Brown,Tech,92000.0,1.0,2018-06-30,Tech,500000,18.4,No,75.2
3,4,Dave,Wilson,Marketing,75000.0,2.0,2020-01-10,Marketing,200000,37.5,Yes,76.0
4,5,Eve,Davis,Tech,98000.0,1.0,2017-09-01,Tech,500000,19.6,No,75.2
5,6,Frank,Miller,Sales,82000.0,2.0,2019-11-15,Sales,300000,27.3,Yes,80.3
6,7,Grace,Moore,Marketing,77000.0,4.0,2020-08-20,Marketing,200000,38.5,Yes,76.0
7,8,Hank,Taylor,Tech,91000.0,3.0,2018-04-12,Tech,500000,18.2,No,75.2
8,9,Iris,Anderson,Sales,79000.0,6.0,2021-02-28,Sales,300000,26.3,Yes,80.3


In [3]:
-- Find all manager-employee pairs and calculate:
-- 1. Salary ratio (employee salary / manager salary)
-- 2. Flag if employee makes more than manager
-- 3. Count how many direct reports each manager has
-- Include manager names and employee names

SyntaxError: invalid syntax (8276941.py, line 1)

In [59]:
emp = employees.copy()
mgr = employees.copy()
emp = emp.merge(mgr, left_on='manager_id', right_on='id', how='inner',  suffixes=('_emp', '_mgr'))
ratio = emp[['first_name_emp','last_name_emp', 'first_name_mgr','last_name_mgr', 'salary_emp', 'salary_mgr', 'id_emp', 'manager_id_emp']]
summary = ratio.copy()
summary['ratio'] = summary['salary_emp'] / summary['salary_mgr']
gommi = summary.groupby('manager_id_emp')['first_name_emp'].count().reset_index(name='direct_reports_count')
summary = summary.merge(gommi, on='manager_id_emp')
summary['flag'] = np.where(summary['salary_emp'] > summary['salary_mgr'], 'Yes', 'No')
summary

,first_name_emp,last_name_emp,first_name_mgr,last_name_mgr,salary_emp,salary_mgr,id_emp,manager_id_emp,ratio,direct_reports_count,flag
0,Bob,Jones,Ana,Smith,80000.0,95000.0,2,1.0,0.842105,3,No
1,Cara,Brown,Ana,Smith,92000.0,95000.0,3,1.0,0.968421,3,No
2,Dave,Wilson,Bob,Jones,75000.0,80000.0,4,2.0,0.937500,2,No
3,Eve,Davis,Ana,Smith,98000.0,95000.0,5,1.0,1.031579,3,Yes
4,Frank,Miller,Bob,Jones,82000.0,80000.0,6,2.0,1.025000,2,Yes
5,Grace,Moore,Dave,Wilson,77000.0,75000.0,7,4.0,1.026667,1,Yes
6,Hank,Taylor,Cara,Brown,91000.0,92000.0,8,3.0,0.989130,1,No
7,Iris,Anderson,Frank,Miller,79000.0,82000.0,9,6.0,0.963415,1,No
8,Jack,Thomas,Grace,Moore,NaN,77000.0,10,7.0,NaN,1,No


In [60]:
    -- Create a report showing:
    -- 1. Employees who are "outliers" (salary > 1.5x dept median OR < 0.5x dept median)
    -- 2. Add columns showing: dept median, how far above/below median (as %)
    -- 3. Only include departments with 3+ employees

SyntaxError: invalid decimal literal (4249807772.py, line 2)

In [17]:
medianbydept = employees.groupby('dept')['salary'].median()
medianbydept
emp = employees.copy()
emp = emp.merge(medianbydept, on='dept', suffixes=('','_median'))
yup = emp[(emp['salary'] > 1.5 * emp['salary_median']) | (emp['salary'] < 0.5 * emp['salary_median'])]
yup['relation_to_median'] = (yup['salary'] - yup['salary_median']) / yup['salary_median'] * 100
counts = yup.merge(emp.groupby('dept')['first_name'].count(), on='dept')
counts = counts[counts['first_name_y'] > 2]
counts

,id,first_name_x,last_name,dept,salary,manager_id,hire_date,salary_median,relation_to_median,first_name_y


In [18]:
-- For each department (ordered by hire date):
-- 1. Show running total of salaries
-- 2. Show running average salary
-- 3. Mark when dept first exceeded 200k in total salaries
-- 4. Include hire_date, first_name, salary in output

SyntaxError: invalid decimal literal (63773972.py, line 4)

In [58]:
depts = employees[['first_name', 'salary', 'dept', 'hire_date']]
depts = depts[depts['salary'].notna()]
depts = depts.sort_values('hire_date')
depts['running_salary'] = depts.groupby('dept')['salary'].cumsum()
depts['running_200k'] = np.where(depts['running_salary'] > 200000, 'Passed', 'Not-Passed')
depts['running_count'] = depts.groupby('dept').cumcount() + 1
depts['running_avg'] = depts['running_salary'] / depts['running_count']
depts
depts['is_first_200k'] = (depts['running_salary'] > 200000) & (depts.groupby('dept')['running_salary'].shift(1) <= 200000)
depts

,first_name,salary,dept,hire_date,running_salary,running_200k,running_count,running_avg,is_first_200k
4,Eve,98000.0,Tech,2017-09-01,98000.0,Not-Passed,1,98000.000000,False
0,Ana,95000.0,Tech,2018-01-15,193000.0,Not-Passed,2,96500.000000,False
7,Hank,91000.0,Tech,2018-04-12,284000.0,Passed,3,94666.666667,True
2,Cara,92000.0,Tech,2018-06-30,376000.0,Passed,4,94000.000000,False
1,Bob,80000.0,Sales,2019-03-22,80000.0,Not-Passed,1,80000.000000,False
5,Frank,82000.0,Sales,2019-11-15,162000.0,Not-Passed,2,81000.000000,False
3,Dave,75000.0,Marketing,2020-01-10,75000.0,Not-Passed,1,75000.000000,False
6,Grace,77000.0,Marketing,2020-08-20,152000.0,Not-Passed,2,76000.000000,False
8,Iris,79000.0,Sales,2021-02-28,241000.0,Passed,3,80333.333333,True


In [59]:
-- Compare all pairs of departments and show:
-- 1. Average salary difference
-- 2. Which dept pays more
-- 3. Ratio of average salaries
-- 4. Exclude self-comparisons and show each pair only once

SyntaxError: invalid syntax (3436215905.py, line 1)

In [93]:
de = employees[['dept', 'salary']]
de = de[de['salary'].notna()]
de = de.groupby('dept').agg({'salary':'mean'})
de = de.reset_index()
de2 = de.copy()
de2 = de.merge(de2,  how='cross')
de2 = de2[(de2['salary_x'] != de2['salary_y']) & (de2['dept_x'] < de2['dept_y'])]
de2['difference'] = abs(de2['salary_y'] - de2['salary_x'])
de2['which_pays_more'] = np.where(de2['salary_y'] - de2['salary_x'] > 0, de2['dept_y'], de2['dept_x'])
de2['ratio'] = de2['salary_x'] / de2['salary_y']

,dept_x,salary_x,dept_y,salary_y,difference,which_pays_more,ratio
1,Marketing,76000.000000,Sales,80333.333333,4333.333333,Sales,0.946058
2,Marketing,76000.000000,Tech,94000.000000,18000.000000,Tech,0.808511
5,Sales,80333.333333,Tech,94000.000000,13666.666667,Tech,0.854610


In [94]:
-- Create a final report that combines everything:
-- For each department show:
-- 1. Basic stats (count, avg/median/std salary)
-- 2. Hiring trends (employees hired per year)
-- 3. Budget efficiency (total salaries / budget)
-- 4. Top 2 earners' names and salaries
-- Present as a single merged DataFrame

SyntaxError: unterminated string literal (detected at line 6) (617256010.py, line 6)

In [241]:
# Select key columns and merge department info into employee data
emp_report = employees[['first_name', 'last_name', 'dept', 'salary', 'hire_date']]
emp_report = emp_report.merge(departments, left_on='dept', right_on='dname')  # merge on dept name to bring in budget
emp_report = emp_report.drop('dname', axis=1)  # drop redundant column after merge
emp_report.columns = ['first_name', 'last_name', 'dept', 'salary', 'hire_date', 'dept_budget']  # rename for clarity

# Aggregate salary statistics per department (group-level summary)
emp_report_grouped = emp_report.groupby('dept').agg(
    avg_salary = ('salary','mean'), 
    median_salary = ('salary','median'), 
    std_salary = ('salary','std'))

# Extract hire year as separate feature for hiring trend analysis
emp_report['hire_year'] = pd.to_datetime(emp_report['hire_date']).dt.year

# Group by (dept, year) and count hires → gives MultiIndex result
counter = emp_report.groupby(['dept','hire_year']).agg(emp_hired_that_year = ('salary','count'))

# Pivot the MultiIndex — converts years to columns, one row per dept
counter = counter.unstack(level='hire_year')

# Flatten the resulting column MultiIndex — makes columns like 'hired_2017', 'hired_2021'
counter.columns = ['hired_' + str(col[1]) for col in counter.columns]

# Replace NaNs from years with no hires in that department with 0
counter = counter.fillna(0)

# Join hiring trend data into summary table; both share 'dept' as index
emp_report_grouped = emp_report_grouped.join(counter)

# Compute total salary paid and total dept budget per department
utilized = emp_report.groupby('dept').agg({'salary':'sum', 'dept_budget':'sum'})

# Join total salary and budget into hiring trend table for further metrics
counter = counter.join(utilized, on='dept')

# Compute budget utilization % from total salary and total budget
counter['utilized_percentage'] = (
    (counter['salary'] / counter['dept_budget'] * 100)
    .round(2).astype(str) + '%'
)

# Select only needed columns, rename salary to total_salary, and join to master summary
extra = counter[['salary', 'dept_budget', 'utilized_percentage']].rename(columns={'salary': 'total_salary'})
emp_report_grouped = emp_report_grouped.join(extra)

# ↓↓↓ Top earners extraction ↓↓↓

# Sort employees by department and descending salary — ensures top earners are first
sorted_emp = emp_report.sort_values(['dept', 'salary'], ascending=[True, False])

# Keep only the top 2 earners per department
top2 = sorted_emp.groupby('dept').head(2).copy()  # head(2) applied within each dept group

# Collapse top 2 earners into one row per dept — builds a summary Series for each group
top_earners = (
    top2.drop(columns='dept')
    .groupby(top2['dept'], group_keys=False)
    .apply(lambda g: pd.Series({
        'top1_name': f"{g.iloc[0]['first_name']} {g.iloc[0]['last_name']}",
        'top1_salary': round(g.iloc[0]['salary']),
        'top2_name': f"{g.iloc[1]['first_name']} {g.iloc[1]['last_name']}" if len(g) > 1 else None,
        'top2_salary': round(g.iloc[1]['salary']) if len(g) > 1 else None
    }))
)


# Merge top earner info into department-level summary
emp_report_grouped = emp_report_grouped.join(top_earners)

# Final result: emp_report_grouped now contains full department summaries with all metrics
emp_report_grouped


,avg_salary,median_salary,std_salary,hired_2017,hired_2018,hired_2019,hired_2020,hired_2021,total_salary,dept_budget,utilized_percentage,top1_name,top1_salary,top2_name,top2_salary
dept,,,,,,,,,,,,,,,
Marketing,76000.000000,76000.0,1414.213562,0.0,0.0,0.0,2.0,0.0,152000.0,400000,38.0%,Grace Moore,77000,Dave Wilson,75000
Sales,80333.333333,80000.0,1527.525232,0.0,0.0,2.0,0.0,1.0,241000.0,900000,26.78%,Frank Miller,82000,Bob Jones,80000
Tech,94000.000000,93500.0,3162.277660,1.0,3.0,0.0,0.0,0.0,376000.0,2000000,18.8%,Eve Davis,98000,Ana Smith,95000


In [293]:
avg_sal = employees[['salary', 'dept']]
avg_sal = avg_sal.groupby('dept').agg({'salary':'mean'})
departments.fillna(0)
avg_sal = departments.merge(avg_sal, 
                            left_on='dname',
                            right_on='dept', 
                            how='left').fillna({'salary_y': 0, 'salary_x': 0}).drop_duplicates('dname')
avg_sal = avg_sal.drop(['salary_x'], axis=1)
avg_sal = avg_sal.rename(columns={'salary_y': 'salary_mean'})
avg_sal = avg_sal.sort_values('salary_mean', ascending=False)
avg_sal

,dname,budget,dept,salary_mean
0,Tech,500000,Tech,94000.000000
4,Sales,300000,Sales,80333.333333
7,Marketing,200000,Marketing,76000.000000
9,HR,150000,NaN,0.000000


In [295]:
/* For each department:
   1. Calculate its median salary (excluding NULLs)
   2. Count how many employees earn ABOVE that median
   3. Count how many earn BELOW (or equal to) that median
   Output: dept | above_median | below_or_equal_median
*/

SyntaxError: invalid syntax (3153537264.py, line 1)

In [331]:
empsal = employees[['salary', 'dept']]
empsal = empsal[(empsal['dept'].notna()) & (empsal['salary'].notna())]
empsal2 = empsal.groupby('dept').agg({'salary':'median'})
empsal = empsal.merge(empsal2, on='dept')
empsal['relation_to_median'] = np.where(empsal['salary_x'] > empsal['salary_y'], 'Above', 'Below OR Equal Median')
empsal = empsal.groupby(['dept', 'relation_to_median']).agg({'relation_to_median':'count'})
empsal = empsal.unstack('relation_to_median')
empsal = (empsal
          .fillna(0)
          .reset_index())
empsal.columns = ['dept', 'above_median', 'below_or_equal_median']
empsal

,dept,above_median,below_or_equal_median
0,Marketing,1,1
1,Sales,1,2
2,Tech,2,2


In [332]:
/* 3-hire rolling average salary by department, ordered by hire_date */
SELECT id, first_name, last_name, dept, salary, hire_date,
       AVG(salary) OVER (
         PARTITION BY dept
         ORDER BY hire_date
         ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
       ) AS rolling_avg_3
FROM   employees
ORDER  BY dept, hire_date;
```

SyntaxError: invalid syntax (3621360985.py, line 1)

In [357]:
emp = employees[['id', 'first_name', 'last_name', 'dept', 'salary', 'hire_date']].copy()
emp = emp[(emp['dept'].notna()) & (emp['salary'].notna())]
emp['hire_date'] = pd.to_datetime(emp['hire_date'])
emp = emp.sort_values(['dept', 'hire_date'])

rolling_avg = (emp
    .groupby('dept')['salary']
    .rolling(window=3, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True))

emp['rolling_avg_3'] = rolling_avg.values
emp

,id,first_name,last_name,dept,salary,hire_date,rolling_avg_3
3,4,Dave,Wilson,Marketing,75000.0,2020-01-10,75000.000000
6,7,Grace,Moore,Marketing,77000.0,2020-08-20,76000.000000
1,2,Bob,Jones,Sales,80000.0,2019-03-22,80000.000000
5,6,Frank,Miller,Sales,82000.0,2019-11-15,81000.000000
8,9,Iris,Anderson,Sales,79000.0,2021-02-28,80333.333333
4,5,Eve,Davis,Tech,98000.0,2017-09-01,98000.000000
0,1,Ana,Smith,Tech,95000.0,2018-01-15,96500.000000
7,8,Hank,Taylor,Tech,91000.0,2018-04-12,94666.666667
2,3,Cara,Brown,Tech,92000.0,2018-06-30,92666.666667


In [358]:
/* Departments whose total salary exceeds 50 % of their budget */
SELECT d.dname                              AS dept,
       SUM(e.salary)                        AS total_salary,
       d.budget,
       SUM(e.salary) / d.budget * 100       AS utilization_pct,
       COUNT(e.id)                          AS employee_count
FROM   departments d
JOIN   employees   e ON e.dept = d.dname
GROUP  BY d.dname, d.budget
HAVING SUM(e.salary) > 0.50 * d.budget
ORDER  BY utilization_pct DESC;

SyntaxError: invalid syntax (347171192.py, line 1)

In [391]:
sd = employees[['salary', 'dept']]
sd = sd.groupby('dept').agg({'salary':'sum', 'dept':'count'}).rename(columns={'salary':'sum_salaries', 'dept': 'emp_count'}).reset_index()
departments2 = departments.merge(sd, left_on='dname', right_on='dept')
departments2['utilization_pct'] = (departments2['sum_salaries'] / departments2['budget']) * 100
departments2 = departments2[departments2['utilization_pct'] > 50]
departments2

,dname,budget,dept,sum_salaries,emp_count,utilization_pct
0,Tech,500000,Tech,376000.0,4,75.200000
1,Sales,300000,Sales,241000.0,3,80.333333
2,Marketing,200000,Marketing,152000.0,2,76.000000


In [392]:
-- Rank employees by salary within dept using percentile_rank
-- Show what percentile each employee falls into (0-100)
-- Handle ties by giving same percentile to same salaries

SyntaxError: invalid syntax (567821924.py, line 1)

In [440]:
emp = employees.copy()
emp = emp[emp['salary'].notna()]
emp['rankInDept'] = emp.groupby('dept')['salary'].rank()
emp['deptCount'] = emp.groupby('dept')['salary'].transform('count')
emp['percentile_%'] = round((emp['rankInDept'] - 1) / (emp['deptCount'] - 1) * 100)
# or to handle single emp depts
#  emp['percentile_%'] = np.where(
#    emp['deptCount'] == 1, 
#    50.0,  # single employee = 50th percentile
#    round((emp['rankInDept'] - 1) / (emp['deptCount'] - 1) * 100)
#)
emp = emp.sort_values(['dept', 'salary'], ascending=[True, False])
emp[['id', 'first_name', 'dept', 'salary', 'percentile_%']]

,id,first_name,dept,salary,percentile_%
6,7,Grace,Marketing,77000.0,100.0
3,4,Dave,Marketing,75000.0,0.0
5,6,Frank,Sales,82000.0,100.0
1,2,Bob,Sales,80000.0,50.0
8,9,Iris,Sales,79000.0,0.0
4,5,Eve,Tech,98000.0,100.0
0,1,Ana,Tech,95000.0,67.0
2,3,Cara,Tech,92000.0,33.0
7,8,Hank,Tech,91000.0,0.0


In [420]:
-- For each employee, calculate:
-- 1. Their salary rank within dept (dense_rank)
-- 2. Difference from dept leader (highest salary)
-- 3. Whether they're in top 50% of dept (Y/N flag)

SyntaxError: unterminated string literal (detected at line 4) (3805971371.py, line 4)

In [452]:
emp = employees.copy()
emp = emp[emp['salary'].notna()]
emp['deptrank'] = emp.groupby('dept')['salary'].rank(ascending=False, method='dense')
emp['biggestInDept'] = emp.groupby('dept')['salary'].transform('max')
emp['difference_compared_to_nr1'] = emp['biggestInDept'] - emp['salary']
emp['median'] = round(emp.groupby('dept')['salary'].transform('median'))
emp['flag'] = np.where(emp['salary'] > emp['median'], 'Y', 'N')
emp

,id,first_name,last_name,dept,salary,manager_id,hire_date,deptrank,biggestInDept,difference_compared_to_nr1,median,flag
0,1,Ana,Smith,Tech,95000.0,NaN,2018-01-15,2.0,98000.0,3000.0,93500.0,Y
1,2,Bob,Jones,Sales,80000.0,1.0,2019-03-22,2.0,82000.0,2000.0,80000.0,N
2,3,Cara,Brown,Tech,92000.0,1.0,2018-06-30,3.0,98000.0,6000.0,93500.0,N
3,4,Dave,Wilson,Marketing,75000.0,2.0,2020-01-10,2.0,77000.0,2000.0,76000.0,N
4,5,Eve,Davis,Tech,98000.0,1.0,2017-09-01,1.0,98000.0,0.0,93500.0,Y
5,6,Frank,Miller,Sales,82000.0,2.0,2019-11-15,1.0,82000.0,0.0,80000.0,Y
6,7,Grace,Moore,Marketing,77000.0,4.0,2020-08-20,1.0,77000.0,0.0,76000.0,Y
7,8,Hank,Taylor,Tech,91000.0,3.0,2018-04-12,4.0,98000.0,7000.0,93500.0,N
8,9,Iris,Anderson,Sales,79000.0,6.0,2021-02-28,3.0,82000.0,3000.0,80000.0,N


In [453]:
-- Compare hiring patterns year-over-year
-- Show: year, total_hired, change_from_prev_year, pct_change

SyntaxError: invalid syntax (2156270913.py, line 1)

In [493]:
emp  = employees.copy()
emp['hire_year'] = pd.to_datetime(emp['hire_date']).dt.year

# Create a summary by year
summary_by_year = emp.groupby('hire_year').size().reset_index(name='total_hired')
summary_by_year['change_from_prev_year'] = summary_by_year['total_hired'] - summary_by_year['total_hired'].shift(1)
summary_by_year['pct_change'] =  (summary_by_year['change_from_prev_year'] /  summary_by_year['total_hired'].shift(1)) * 100
summary_by_year
# emp = employees.copy()
# emp = emp[emp['hire_date'].notna()]
# emp['hire_year'] = pd.to_datetime(emp['hire_date']).dt.year
# emp['countbydept'] = emp.groupby('hire_year')['id'].transform('count')
# emp['countbyYear'] = emp.groupby('hire_year')['countbydept'].transform('sum')
# emp = emp.sort_values('hire_year')
# emp

,hire_year,total_hired,change_from_prev_year,pct_change
0,2017,1,NaN,NaN
1,2018,3,2.0,200.000000
2,2019,2,-1.0,-33.333333
3,2020,2,0.0,0.000000
4,2021,1,-1.0,-50.000000
5,2022,1,0.0,0.000000


In [494]:
-- Find all manager-subordinate pairs where:
-- 1. Manager has 2+ direct reports
-- 2. Calculate avg salary of all reports for each manager
-- 3. Flag if any report earns within 10% of manager

SyntaxError: invalid syntax (172249200.py, line 1)

In [587]:
emp = employees.copy()
managers = employees.copy()

count = managers.groupby('manager_id').size().reset_index(name='count')
count = count.set_index('manager_id')['count']

managers['count'] = managers['id'].map(count)
managers = managers[managers['count'] >= 2]
managers = managers.merge(employees, left_on='id', right_on='manager_id')
meansalary = managers.groupby('id_x').agg({'salary_y':'mean'})
managers['mean_salary_of_reports'] = managers['id_x'].map(meansalary['salary_y'])
managers['is_high_earner'] = (managers['salary_y'] >= managers['salary_x'] * 0.9).astype(int)
stats = managers.groupby('id_x').agg({
    'salary_y':'mean',
    'is_high_earner':'max'
})
managers['mean_salary_of_reports'] = managers['id_x'].map(stats['salary_y'])
managers['has_high_earner'] = managers['id_x'].map(stats['is_high_earner'])
managers[['id_x', 'first_name_x', 'last_name_x', 'salary_x',
       'manager_id_x', 'id_y', 'first_name_y',
       'last_name_y', 'salary_y', 'manager_id_y',
       'mean_salary_of_reports', 'is_high_earner', 'has_high_earner']]

,id_x,first_name_x,last_name_x,salary_x,manager_id_x,id_y,first_name_y,last_name_y,salary_y,manager_id_y,mean_salary_of_reports,is_high_earner,has_high_earner
0,1,Ana,Smith,95000.0,NaN,2,Bob,Jones,80000.0,1.0,90000.0,0,1
1,1,Ana,Smith,95000.0,NaN,3,Cara,Brown,92000.0,1.0,90000.0,1,1
2,1,Ana,Smith,95000.0,NaN,5,Eve,Davis,98000.0,1.0,90000.0,1,1
3,2,Bob,Jones,80000.0,1.0,4,Dave,Wilson,75000.0,2.0,78500.0,1,1
4,2,Bob,Jones,80000.0,1.0,6,Frank,Miller,82000.0,2.0,78500.0,1,1


In [588]:
-- For each department calculate:
-- 1. Current utilization (total_salaries / budget)
-- 2. Projected utilization if everyone gets 10% raise
-- 3. How many more people could be hired at avg dept salary

SyntaxError: invalid syntax (2744331891.py, line 1)

In [698]:
# Create copies
temp = employees.copy()
dep = departments.copy()
emp_avg = temp['salary'].mean()
# Create 
summary_table = temp.groupby('dept').agg(salary_total = ('salary','sum') , salary_avg=('salary','mean'))
dep = dep.merge(summary_table, left_on='dname', right_on='dept', how='left')
summary_table = dep
temp['salary'] = temp['salary'] * 1.1
temp = temp.groupby('dept').agg(
    salarytotal_plus10percent=('salary','sum'), 
)
summary_table = summary_table.merge(temp, left_on='dname', right_on='dept', how='left')

summary_table['salary_total'] = summary_table['salary_total'].fillna(0)
summary_table['salary_avg'] = summary_table['salary_avg'].fillna(emp_avg)
summary_table['salarytotal_plus10percent'] = summary_table['salarytotal_plus10percent'].fillna(0)

summary_table['utilization_%'] = (summary_table['salary_total'] / summary_table['budget']) * 100
summary_table['projected_util_%'] = (summary_table['salarytotal_plus10percent'] / summary_table['budget']) * 100
summary_table['numberOfEmployeesAvailableToAdd'] = np.where(((summary_table['budget'] - summary_table['salary_total']) / summary_table['salary_avg']) < 1, 0, np.floor(((summary_table['budget'] - summary_table['salary_total']) / summary_table['salary_avg'])))
summary_table

,dname,budget,salary_total,salary_avg,salarytotal_plus10percent,utilization_%,projected_util_%,numberOfEmployeesAvailableToAdd
0,Tech,500000,376000.0,94000.000000,413600.0,75.200000,82.720000,1.0
1,Sales,300000,241000.0,80333.333333,265100.0,80.333333,88.366667,0.0
2,Marketing,200000,152000.0,76000.000000,167200.0,76.000000,83.600000,0.0
3,HR,150000,0.0,85444.444444,0.0,0.000000,0.000000,1.0


In [699]:
-- Group employees by hire year (cohort) and analyze:
-- 1. Cohort size
-- 2. Current avg salary by cohort
-- 3. Salary growth rate (current avg / starting avg)
-- Assume 3% annual raises for starting salary calculation

SyntaxError: invalid syntax (655157818.py, line 1)

In [774]:
df = employees.dropna(subset=['salary']).copy()

#Employee-level enrichments first
df['hire_year'] = pd.to_datetime(df['hire_date']).dt.year
df['years_employed'] = 2025 - df['hire_year']
df['starting_salary'] = df['salary'] / 1.03 ** df['years_employed']

#Cohort-level aggregation
cohort_agg = df.groupby('hire_year').agg(
    cohort_size = ('first_name', 'count'),
    current_avg = ('salary', 'mean'),
    starting_avg = ('starting_salary', 'mean')
)

#Compute growth rate at the cohort level
cohort_agg['salary_growth_rate'] = (cohort_agg['current_avg'] / cohort_agg['starting_avg']).round(2)
cohort_agg

,cohort_size,current_avg,starting_avg,salary_growth_rate
hire_year,,,,
2017,1,98000.000000,77362.104963,1.27
2018,3,92666.666667,75346.480051,1.23
2019,2,81000.000000,67836.224791,1.19
2020,2,76000.000000,65558.267613,1.16
2021,1,79000.000000,70190.476785,1.13


In [775]:
-- Build complete org hierarchy showing:
-- employee -> direct_manager -> skip_manager -> dept_head
-- Include level in hierarchy (1=dept_head, 2=direct report to head, etc)   

SyntaxError: invalid syntax (1971232306.py, line 1)

In [21]:
skip = employees.copy()
skip['full_name'] = skip['first_name'] + ' ' + skip['last_name']
skip = skip[['id', 'full_name', 'manager_id']]
emptree = skip.copy().fillna(0)

emptree['divider0'] = '|'
# manager merge and rename

emptree = emptree.merge(skip, left_on='manager_id', right_on='id', suffixes=('_emp', '_mgr'), how='left')

emptree.rename(columns={
    'id_mgr':'direct_manager_id',
    'full_name_mgr': 'direct_manager_name',
    'manager_id_mgr': 'direct_manager_manager_id'
}, inplace=True)

# Divide
emptree['divider1'] = '|'
#skip manager merge and rename

emptree = emptree.merge(skip, left_on='direct_manager_manager_id', right_on='id' ,how='left')

emptree.rename(columns={
    'id': 'skip_manager_id',
    'full_name': 'skip_manager_name',
    'manager_id': 'skip_manager_manager_id'
}, inplace=True)


# Divide
emptree['divider2'] = '|'

# #dept heads
emptree  = emptree.merge(skip, left_on='skip_manager_manager_id', right_on='id', how='left')

emptree.rename(columns={
    'id': 'dept_head_id',
    'full_name': 'dept_head_name',
    'manager_id': 'dept_head_manager_id'
}, inplace=True)

emptree = emptree.merge(skip, left_on='dept_head_manager_id', right_on='id', how='left')
emptree.rename(columns={
    'id':'org_head_id',
    'full_name': 'org_head_name',
    'manager_id': 'org_head_manager_id'
}, inplace=True)

emptree['level_in_hierarchy'] = emptree[['direct_manager_id','skip_manager_id','dept_head_id','org_head_id']].notna().sum(axis=1) + 1

emptree

,id_emp,full_name_emp,manager_id_emp,divider0,direct_manager_id,direct_manager_name,direct_manager_manager_id,divider1,skip_manager_id,skip_manager_name,skip_manager_manager_id,divider2,dept_head_id,dept_head_name,dept_head_manager_id,org_head_id,org_head_name,org_head_manager_id,level_in_hierarchy
0,1,Ana Smith,0.0,|,NaN,NaN,NaN,|,NaN,NaN,NaN,|,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2,Bob Jones,1.0,|,1.0,Ana Smith,NaN,|,NaN,NaN,NaN,|,NaN,NaN,NaN,NaN,NaN,NaN,2
2,3,Cara Brown,1.0,|,1.0,Ana Smith,NaN,|,NaN,NaN,NaN,|,NaN,NaN,NaN,NaN,NaN,NaN,2
3,4,Dave Wilson,2.0,|,2.0,Bob Jones,1.0,|,1.0,Ana Smith,NaN,|,NaN,NaN,NaN,NaN,NaN,NaN,3
4,5,Eve Davis,1.0,|,1.0,Ana Smith,NaN,|,NaN,NaN,NaN,|,NaN,NaN,NaN,NaN,NaN,NaN,2
5,6,Frank Miller,2.0,|,2.0,Bob Jones,1.0,|,1.0,Ana Smith,NaN,|,NaN,NaN,NaN,NaN,NaN,NaN,3
6,7,Grace Moore,4.0,|,4.0,Dave Wilson,2.0,|,2.0,Bob Jones,1.0,|,1.0,Ana Smith,NaN,NaN,NaN,NaN,4
7,8,Hank Taylor,3.0,|,3.0,Cara Brown,1.0,|,1.0,Ana Smith,NaN,|,NaN,NaN,NaN,NaN,NaN,NaN,3
8,9,Iris Anderson,6.0,|,6.0,Frank Miller,2.0,|,2.0,Bob Jones,1.0,|,1.0,Ana Smith,NaN,NaN,NaN,NaN,4
9,10,Jack Thomas,7.0,|,7.0,Grace Moore,4.0,|,4.0,Dave Wilson,2.0,|,2.0,Bob Jones,1.0,1.0,Ana Smith,NaN,5


In [881]:
-- Create matrix showing salary comparisons between all dept pairs
-- For each pair show: avg_salary_ratio, median_salary_ratio

SyntaxError: invalid syntax (1818874678.py, line 1)

In [920]:
# group by dept and aggregate for mean salary and median
emp1 = employees.copy()
emp1 = emp1.groupby('dept').agg(mean_sal = ('salary','mean'), median_sal = ('salary','median'))
# add the dept as a column instead of an index
emp1 = emp1.reset_index()
# cross merge to get cartesian product
emp2 = emp1.copy()
emp1 = emp1.merge(emp2, how='cross')
# filter for not the same depts replicated(tech/tech, or tech/sales AND sales/tech)
emp1 = emp1[(emp1['dept_x'] < emp1['dept_y'])]
emp1['avg_salary_ratio'] = emp1['mean_sal_x'] / emp1['mean_sal_y']
emp1['median_salary_ratio'] = emp1['median_sal_x'] / emp1['median_sal_y']
emp1 = emp1.pivot_table(
    index='dept_x',
    columns='dept_y',
    values=['avg_salary_ratio', 'median_salary_ratio']
)
emp1

avg_salary_ratio           median_salary_ratio          
dept_y               Sales      Tech               Sales      Tech
dept_x                                                            
Marketing         0.946058  0.808511                0.95  0.812834
Sales                  NaN  0.854610                 NaN  0.855615

In [921]:
-- Create salary bands (0-70k, 70-85k, 85-100k, 100k+)
-- For each band show:
-- 1. Employee count and percentage of total
-- 2. Which departments are represented
-- 3. Avg years of experience (from hire_date)  

SyntaxError: invalid decimal literal (1918862257.py, line 1)

In [34]:
emp1 = employees.copy()
emp1 = emp1[emp1['salary'].notna()]
total_count = emp1['salary'].count()
emp1 = emp1[emp1['dept'].notna()]
labels = ["0-70k", "70-85k", "85-100k", "100k+"]
emp1['bin'] = pd.cut(emp1['salary'], bins=[0,70000,85000,100000,np.inf], labels=labels)
emp1['yoe'] = 2025 - pd.to_datetime(emp1['hire_date']).dt.year
emp1 = emp1.groupby('bin').agg({'salary':'count', 
                                'yoe':'mean', 
                               'dept': lambda x: ", ".join(set(x)) })
emp1['percentage'] = round(emp1['salary'] / total_count *100)
emp1

/var/folders/pn/q39n4_bd7rq84krr4s_56sn00000gn/T/ipykernel_41885/540454113.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  emp1 = emp1.groupby('bin').agg({'salary':'count',


,salary,yoe,dept,percentage
bin,,,,
0-70k,0,NaN,,0.0
70-85k,5,5.20,"Sales, Marketing",56.0
85-100k,4,7.25,Tech,44.0
100k+,0,NaN,,0.0


In [14]:
-- Simulate performance ratings based on:
-- 1. Salary relative to dept average (40% weight)
-- 2. Tenure/experience (30% weight)
-- 3. Position in hierarchy (30% weight)
-- Create rating 1–5
-- Optional: enforce per-department raise budget

SyntaxError: invalid character '–' (U+2013) (1048855297.py, line 5)

In [49]:
emp = employees.copy()
emp = emp[emp['salary'].notna()]
emp = emp[emp['dept'].notna()]

emp2 = emp.copy()
emp2 = emp2.groupby('dept')['salary'].agg(min_salary='min',max_salary='max')
emp = emp.merge(emp2, on='dept')

emp['yoe'] = 2025 - pd.to_datetime(emp['hire_date']).dt.year

emp3 = emp.copy()
emp3 = emp3.groupby('dept')['yoe'].agg(min_yoe='min', max_yoe='max')
emp = emp.merge(emp3, on='dept')

emptree = emptree[['id_emp', 'level_in_hierarchy']]
emp = emp.merge(emptree, left_on='id', right_on='id_emp')
    
emp4 = emp.copy()
emp4 = emp4.groupby('dept')['level_in_hierarchy'].agg(min_lvl='min', max_lvl='max')
emp = emp.merge(emp4, on='dept')

emp['normalized_salary'] = (emp['salary'] - emp['min_salary']) / (emp['max_salary'] - emp['min_salary'])
emp.loc[emp['max_salary'] == emp['min_salary'], 'normalized_salary'] = 0.5  # flat dept = midpoint
emp['normalized_yoe'] = (emp['yoe'] - emp['min_yoe']) / (emp['max_yoe'] - emp['min_yoe'])
emp.loc[emp['max_yoe'] == emp['min_yoe'], 'normalized_yoe'] = 0.5
emp['normalized_position'] = 1 - ((emp['level_in_hierarchy'] - emp['min_lvl']) / (emp['max_lvl'] - emp['min_lvl']))
emp.loc[emp['max_lvl'] == emp['min_lvl'], 'normalized_position'] = 0.5
emp['performance'] = 0.4 * emp['normalized_salary'] + 0.3 * emp['normalized_yoe'] + 0.3 * emp['normalized_position']

final_table = emp[['normalized_salary', 'normalized_yoe', 'normalized_position', 'performance']].copy()
final_table['rating'] = pd.cut(
    final_table['performance'],
    bins=[-float("inf"), 0.2, 0.4, 0.6, 0.8, float("inf")],
    labels=[1, 2, 3, 4, 5]
)
final_table

,normalized_salary,normalized_yoe,normalized_position,performance,rating
0,0.571429,0.0,1.0,0.528571,3
1,0.333333,1.0,1.0,0.733333,4
2,0.142857,0.0,0.5,0.207143,2
3,0.000000,0.5,1.0,0.450000,3
4,1.000000,1.0,0.5,0.850000,5
5,1.000000,1.0,0.5,0.850000,5
6,1.000000,0.5,0.0,0.550000,3
7,0.000000,0.0,0.0,0.000000,1
8,0.000000,0.0,0.0,0.000000,1


In [50]:
/* Hire heat-map: rows = dept, cols = hire_year, values = hires */
SELECT dept,
       EXTRACT(YEAR FROM hire_date) AS hire_year,
       COUNT(*)                     AS hires
FROM   employees
GROUP  BY dept, EXTRACT(YEAR FROM hire_date)
ORDER  BY dept, hire_year;

SyntaxError: invalid syntax (480688317.py, line 1)

In [115]:
emp = employees.copy()
emp['hire_year'] = pd.to_datetime(emp['hire_date']).dt.year
emp = emp[['dept', 'hire_year']]
emp2 = emp.groupby(['dept', 'hire_year']).agg(count=('dept','count'))
emp2 = emp2.reset_index()
pvt = emp2.pivot_table(
    index='dept',
    columns='hire_year',
    values='count'
)
pvt.fillna(0)

hire_year,2017,2018,2019,2020,2021
dept,,,,,
Marketing,0.0,0.0,0.0,2.0,0.0
Sales,0.0,0.0,2.0,0.0,1.0
Tech,1.0,3.0,0.0,0.0,0.0


In [116]:
/* Running cumulative salary within each dept with lag + crossing flag */
WITH cume AS (
  SELECT id, first_name, dept, salary,
         SUM(salary) OVER (PARTITION BY dept
                           ORDER BY hire_date)              AS dept_cumsum
  FROM   employees
  WHERE  salary IS NOT NULL
)
SELECT *,
       LAG(dept_cumsum) OVER (PARTITION BY dept
                              ORDER BY dept_cumsum) AS dept_cumsum_lag1,
       CASE WHEN dept_cumsum >= 250000
            AND COALESCE(LAG(dept_cumsum) OVER (
                          PARTITION BY dept ORDER BY dept_cumsum),0) < 250000
            THEN 1 ELSE 0 END                         AS crossed_250k
FROM   cume
ORDER  BY dept, dept_cumsum;

SyntaxError: invalid syntax (2048159720.py, line 1)

In [136]:
emp = employees.copy()
emp = emp[emp['salary'].notna()]
emp = emp.sort_values(['dept', 'hire_date'])
emp['dept_cumsum'] = emp.groupby('dept')['salary'].cumsum()
emp['over250k'] = (emp['dept_cumsum'] > 250000) & (emp.groupby('dept')['dept_cumsum'].shift(1) <= 250000)
emp

,id,first_name,last_name,dept,salary,manager_id,hire_date,dept_cumsum,over250k
3,4,Dave,Wilson,Marketing,75000.0,2.0,2020-01-10,75000.0,False
6,7,Grace,Moore,Marketing,77000.0,4.0,2020-08-20,152000.0,False
1,2,Bob,Jones,Sales,80000.0,1.0,2019-03-22,80000.0,False
5,6,Frank,Miller,Sales,82000.0,2.0,2019-11-15,162000.0,False
8,9,Iris,Anderson,Sales,79000.0,6.0,2021-02-28,241000.0,False
4,5,Eve,Davis,Tech,98000.0,1.0,2017-09-01,98000.0,False
0,1,Ana,Smith,Tech,95000.0,NaN,2018-01-15,193000.0,False
7,8,Hank,Taylor,Tech,91000.0,3.0,2018-04-12,284000.0,True
2,3,Cara,Brown,Tech,92000.0,1.0,2018-06-30,376000.0,False


In [137]:
/* Top-3 earners per department aggregated into one row */
WITH ranked AS (
  SELECT e.*,
         ROW_NUMBER() OVER (PARTITION BY dept ORDER BY salary DESC) AS rn
  FROM   employees e
  WHERE  salary IS NOT NULL
)
SELECT dept,
       STRING_AGG(first_name, ', ')         AS top3_names,
       SUM(salary)                          AS top3_total
FROM   ranked
WHERE  rn <= 3
GROUP  BY dept
ORDER  BY top3_total DESC;

SyntaxError: invalid character '’' (U+2019) (1243771618.py, line 1)

In [166]:
emp = employees.copy()
emp = emp[emp['salary'].notna()]
emp = emp.sort_values(['dept', 'salary'], ascending=[True, False])
emp = emp.groupby('dept').head(3)

emp = emp.groupby('dept').agg({'salary':'sum', 'first_name': lambda x: ", ".join(x)})
emp = emp.rename(columns={'first_name': 'top3_names', 'salary':'top3_salary'})
emp

,top3_salary,top3_names
dept,,
Marketing,152000.0,"Grace, Dave"
Sales,241000.0,"Frank, Bob, Iris"
Tech,285000.0,"Eve, Ana, Cara"


In [167]:
/* Managers whose highest-paid report earns more than the manager */
WITH reports AS (
  SELECT m.id              AS manager_id,
         m.first_name      AS manager_name,
         m.salary          AS manager_salary,
         r.first_name      AS report_name,
         r.salary          AS report_salary,
         r.dept
  FROM   employees r
  JOIN   employees m ON r.manager_id = m.id
  WHERE  r.salary IS NOT NULL
)
SELECT manager_name,
       manager_salary,
       report_name         AS top_report_name,
       report_salary       AS top_report_salary,
       report_salary - manager_salary AS salary_gap,
       dept
FROM (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY manager_id ORDER BY report_salary DESC) AS rn
  FROM   reports
) t
WHERE  rn = 1
  AND  report_salary > manager_salary
ORDER BY salary_gap DESC;

SyntaxError: invalid syntax (972090725.py, line 1)

In [222]:
emp = employees.copy()
emp = emp[emp['salary'].notna()]
managers = emp.copy()
managers = managers.merge(emp, how='left', left_on='id', right_on='manager_id', suffixes=['_mgr', '_emp'])
managers = managers.sort_values(['id_mgr', 'salary_emp'], ascending=[True, False])
managers = managers.groupby('id_mgr').head(1)
managers = managers[managers['first_name_emp'].notna()]
managers = managers[managers['salary_emp'] > managers['salary_mgr']]
managers = managers[['first_name_mgr', 'last_name_mgr', 'salary_mgr', 'first_name_emp',
       'last_name_emp', 'salary_emp']]
managers['salary_gap'] = managers['salary_emp'] - managers['salary_mgr']
managers

,first_name_mgr,last_name_mgr,salary_mgr,first_name_emp,last_name_emp,salary_emp,salary_gap
2,Ana,Smith,95000.0,Eve,Davis,98000.0,3000.0
4,Bob,Jones,80000.0,Frank,Miller,82000.0,2000.0
6,Dave,Wilson,75000.0,Grace,Moore,77000.0,2000.0
